# Chapter 18: Sensitivity Analysis for the Average Causal Effect with Unmeasured Confounding

In [1]:
import numpy as np
import pandas as pd
from utils import *

np.random.seed(42)
%load_ext autoreload
%autoreload 1

%load_ext watermark
%watermark --iversions

pandas           : 2.0.3
graphviz         : 0.20.1
matplotlib_inline: 0.1.6
numpy            : 1.24.3



In [2]:
from sklearn.linear_model import LogisticRegression, LinearRegression


def OS_est(
    z,
    y,
    x,
    omod=LinearRegression(),
    pmod=LogisticRegression(C=1e4),
    lb=0,
    ub=1,
    e1=1,
    e0=1,
):
    pscore = pmod.fit(x, z).predict_proba(x)[:, 1]
    pscore = np.clip(pscore, lb, ub)
    # fitted potential outcomes
    outcome1 = omod.fit(x[z == 1, :], y[z == 1]).predict(x)
    outcome0 = omod.fit(x[z == 0, :], y[z == 0]).predict(x)

    ## outcome regression estimator
    ace_reg = (
        np.mean(z * y)
        + np.mean((1 - z) * outcome1 / e1)
        - np.mean(z * outcome0 * e0)
        - np.mean((1 - z) * y)
    )
    ## IPW estimators
    w1 = pscore + (1 - pscore) / e1
    w0 = pscore * e0 + (1 - pscore)
    ace_ipw0 = np.mean(z * y * w1 / pscore) - np.mean((1 - z) * y * w0 / (1 - pscore))
    ace_ipw = np.mean(z * y * w1 / pscore) / np.mean(z / pscore) - np.mean(
        (1 - z) * y * w0 / (1 - pscore)
    ) / np.mean((1 - z) / (1 - pscore))
    ## doubly robust estimator
    aug = outcome1 / pscore / e1 + outcome0 * e0 / (1 - pscore)
    ace_dr = ace_ipw0 + np.mean((z - pscore) * aug)

    return np.array([ace_reg, ace_ipw0, ace_ipw, ace_dr])

In [3]:
nhanes_bmi = pd.read_csv("nhanes_bmi.csv").iloc[:, 1:]
z, y, x = (
    nhanes_bmi.School_meal.values,
    nhanes_bmi.BMI.values,
    nhanes_bmi.iloc[:, 2:].values,
)
x = (x - x.mean(0)) / x.std(0)

In [4]:
E1 = np.array([1 / 2, 1 / 1.7, 1 / 1.5, 1 / 1.3, 1, 1.3, 1.5, 1.7, 2])
E0 = E1.copy()
est = np.zeros((len(E1), len(E0)))

for i in range(len(E1)):
    for j in range(len(E0)):
        est[i, j] = OS_est(z, y, x, e1=E1[i], e0=E0[j])[3]

In [5]:
pd.DataFrame(est, columns=E0, index=E1).round(2)

,0.500000,0.588235,0.666667,0.769231,1.000000,1.300000,1.500000,1.700000,2.000000
0.500000,11.62,10.44,9.40,8.03,4.96,0.97,-1.69,-4.35,-8.35
0.588235,9.22,8.05,7.00,5.64,2.57,-1.42,-4.08,-6.75,-10.74
0.666667,7.63,6.45,5.41,4.04,0.97,-3.02,-5.68,-8.34,-12.33
0.769231,6.03,4.86,3.81,2.45,-0.62,-4.61,-7.27,-9.94,-13.93
1.000000,3.64,2.47,1.42,0.06,-3.01,-7.01,-9.67,-12.33,-16.32
1.300000,1.80,0.63,-0.42,-1.78,-4.85,-8.85,-11.51,-14.17,-18.16
1.500000,0.98,-0.19,-1.24,-2.60,-5.67,-9.66,-12.33,-14.99,-18.98
1.700000,0.36,-0.82,-1.86,-3.23,-6.30,-10.29,-12.95,-15.61,-19.60
2.000000,-0.35,-1.52,-2.57,-3.93,-7.00,-10.99,-13.65,-16.32,-20.31
